In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from ALLCools.clustering import cluster_enriched_features

from ALLCools.plot import *
import scanpy as sc

In [ ]:
data_dir = '/home/qz/hanliu/cemba3c/merfish/202205231554_MouseSagittalM3S1_VMSC01101/output/region_0'

In [ ]:
adata  = sc.read_h5ad('./adata/0809_pretrained_model_202205231554_MouseSagittalM3S1_VMSC01101_QC.h5ad')


# allcools CEF

In [ ]:
cluster_enriched_features(adata,
                          cluster_col='leiden',
                              top_n=20,
                              alpha=0.05,
                              stat_plot=True,
                              method="rna")

In [ ]:
qvals = pd.DataFrame(
    adata.uns['leiden_feature_enrichment']['qvals'],
    index=adata.var_names,
    columns=adata.uns['leiden_feature_enrichment']['cluster_order'])

score = pd.DataFrame(
    adata.uns['leiden_feature_enrichment']['enrichment'],
    index=adata.var_names,
    columns=adata.uns['leiden_feature_enrichment']['cluster_order'])

In [ ]:
# qvals.to_csv('cef_qvals.csv', index = True, header = True)
# score.to_csv('cef_score.csv', index = True, header = True)

In [ ]:
cluster_list = adata.obs['leiden'].unique().sort_values().to_list()

cef_dict = {}
n_top = 10

for cluster in cluster_list:
    cef_dict[cluster] = score[cluster].sort_values(ascending= False)[:n_top]

In [ ]:
score = score.fillna(0)

In [ ]:
score

In [ ]:
all_cef = []
for cluster, top_genes in cef_dict.items():
    all_cef += top_genes

In [ ]:
genes_freq = pd.DataFrame(index = adata.var_names, data = {'top10_freq':pd.Series(all_cef).value_counts()})
genes_freq = genes_freq.fillna(0)

In [ ]:
genes_freq[genes_freq['top10_freq'] == 0]

In [ ]:
genes_freq.to_csv('top10_CEF.csv', index= True, header= True)

# scanpy 

In [ ]:
del adata.uns['log1p']

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')

In [ ]:
sc_top5 =  pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)
sc_top5.to_csv('./tmp/sc_top5.csv', index = None, header = True)

In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=5, sharey=False)